In [11]:
#%pip install openai
from openai import OpenAI
from pathlib import Path
#%pip install pandas
import pandas as pd
import numpy as np
import json
import os
import re

In [12]:
# Paths
context_file_path = "./models/context_files/context.txt"
functionality_map_path = "./models/context_files/functionality_map.json"
test_output_dir = "./models/output/tests"
contextMapped_path = "./models/context_files/contextMapped.txt"  # Define the path for contextMapped

# Create output directories if they don't exist
os.makedirs("./models/output", exist_ok=True)
os.makedirs("./models/context_files", exist_ok=True)
os.makedirs(test_output_dir, exist_ok=True)

In [13]:
controllers = os.listdir("../code/src/main/java/com/bank/controllers")
entities = os.listdir("../code/src/main/java/com/bank/models")
repositories = os.listdir("../code/src/main/java/com/bank/repositories")
services = os.listdir("../code/src/main/java/com/bank/services")
app_path = "../code/src/main/java/com/bank/BankingApiApplication.java"
context_path = "../code/src/main/resources/application.properties"

In [14]:
client = OpenAI(
    api_key = "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",
    base_url = "https://openrouter.ai/api/v1",
)

In [15]:
def get_context(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        file = f.read()
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": "Read me this file of my java springboot application in the variable " + file + " and describe the functionality and context of the code written in the file"
            }
        ],
        model="deepseek/deepseek-r1-distill-llama-70b:free",
        stream=False,
    )
    return chat_completion.choices[0].message.content

In [16]:
def parse_context_file(context_file_path):
    with open(context_file_path, "r", encoding="utf-8") as f:
        content = f.read()
    
    # Initialize variables to track the current section and file
    sections = ["CONTROLLER CONTEXT", "ENTITY/MODEL CONTEXT", "REPOSITORY CONTEXT", "SERVICE CONTEXT", 
                "APPLICATION CONTEXT", "APPLICATION PROPERTIES CONTEXT", "APPLICATION PROPERTIES CONTEXT"]
    
    file_contexts = {}
    current_section = None
    current_file = None
    current_context = []
    
    # Process the file line by line
    lines = content.split("\n")
    for line in lines:
        # Check if this line starts a new section
        if any(line.strip() == section for section in sections):
            current_section = line.strip()
            continue
            
        # Check if this line starts a new file within a section
        for prefix in ["CONTROLLER:", "ENTITY:", "REPOSITORY:", "SERVICE:"]:
            if line.strip().startswith(prefix):
                # Save previous file context if exists
                if current_file and current_context:
                    file_contexts[current_file] = "\n".join(current_context)
                    current_context = []
                
                # Start new file
                current_file = line.strip()[len(prefix):].strip()
                break
        else:
            # If not a section or file header, add to current context
            if current_file:
                current_context.append(line)
    
    # Save the last file context
    if current_file and current_context:
        file_contexts[current_file] = "\n".join(current_context)
    
    return file_contexts

In [17]:
# Initialize context output content
context_output_content = "CONTROLLER CONTEXT\n\n"
for controller in controllers:
    context_output_content += "CONTROLLER: " + controller + "\n\n"
    context_output_content += get_context("../code/src/main/java/com/bank/controllers/" + controller) + "\n\n"

context_output_content += "ENTITY CONTEXT\n\n"
for entity in entities:
    context_output_content += "ENTITY: " + entity + "\n\n"
    context_output_content += get_context("../code/src/main/java/com/bank/models/" + entity) + "\n\n"

context_output_content += "REPOSITORY CONTEXT\n\n"
for repository in repositories:
    context_output_content += "REPOSITORY: " + repository + "\n\n"
    context_output_content += get_context("../code/src/main/java/com/bank/repositories/" + repository) + "\n\n"

context_output_content += "SERVICE CONTEXT\n\n"
for service in services:
    context_output_content += "SERVICE: " + service + "\n\n"
    context_output_content += get_context("../code/src/main/java/com/bank/services/" + service) + "\n\n"

context_output_content += "APPLICATION CONTEXT\n\n"
context_output_content += get_context(app_path) + "\n\n"

context_output_content += "APPLICATION PROPERTIES CONTEXT\n\n"
context_output_content += get_context(context_path) + "\n\n"

# Write to contextMapped file
with open(contextMapped_path, "w", encoding="utf-8") as f:
    f.write(context_output_content)

# Store raw context in context_file_path for use with create_functionality_map
with open(context_file_path, "w", encoding="utf-8") as f:
    f.write(context_output_content)

In [18]:
def create_functionality_map(file_contexts):
    """
    Analyze file contexts to create a functionality map
    """
    # Create context string for the AI prompt
    context_summary = ""
    for file, context in file_contexts.items():
        # Limit context to avoid token limits
        summary = context[:1000] + "..." if len(context) > 1000 else context
        context_summary += f"File: {file}\nContext:\n{summary}\n\n"
    
    # Prompt for the AI to create a functionality map
    prompt = f"""
    Analyze the following file contexts from a Java Spring Boot application and identify all business functionalities.
    For each functionality, list the files that implement it.
    
    {context_summary}
    
    Create a JSON object with the following structure:
    {{
        "functionalities": [
            {{
                "name": "functionality_name",
                "description": "description of what this functionality does",
                "files": ["file_path1", "file_path2"],
                "primary_files": ["main_file_path"],
                "supporting_files": ["supporting_file_path1", "supporting_file_path2"],
                "category": "category (e.g., 'core', 'security', 'data')",
                "complexity": "high/medium/low",
                "test_priority": "high/medium/low"
            }},
            ...
        ]
    }}
    
    Group files by actual business functionality (like "User Authentication", "Journal Entry Management", etc.)
    """
    
    response = client.chat.completions.create(
        messages=[
            {
                "role": "system", 
                "content": "You are an expert Java analyzer that identifies business functionalities from code context."
            },
            {
                "role": "user",
                "content": prompt
            }
        ],
        model="deepseek/deepseek-r1-distill-llama-70b:free",
        stream=False,
    )
    
    # Extract JSON from response
    response_text = response.choices[0].message.content
    
    # Try to find JSON in the response
    json_start = response_text.find("{")
    json_end = response_text.rfind("}")
    
    if json_start >= 0 and json_end > json_start:
        json_str = response_text[json_start:json_end+1]
        try:
            return json.loads(json_str)
        except json.JSONDecodeError:
            # If direct parsing fails, try to clean up the JSON string
            # This handles some common issues like unescaped newlines
            cleaned_json = re.sub(r'(?<!\\)"(?=(,|\s*}|\s*]|\n))', '\\"', json_str)
            try:
                return json.loads(cleaned_json)
            except json.JSONDecodeError:
                print("Error parsing JSON response. Using fallback parsing.")
                # Fallback to a more robust but simplistic parsing approach
                return {"functionalities": [{"name": "Generic Functionality", "files": list(file_contexts.keys())}]}
    else:
        print("Could not find JSON in response. Using empty functionality map.")
        return {"functionalities": []}

In [19]:
def generate_test_cases(functionality_map):
    """
    Generate test cases for each functionality in the map
    """
    functionalities = functionality_map.get("functionalities", [])
    
    for functionality in functionalities:
        functionality_name = functionality["name"]
        sanitized_name = functionality_name.replace(" ", "_").lower()
        print(f"Generating tests for {functionality_name}...")
        
        # Create file names
        feature_file_name = f"{sanitized_name}.feature"
        step_def_file_name = f"{sanitized_name}_steps.java"
        
        # Get the file contexts for this functionality
        file_list = functionality.get("files", [])
        contexts = {}
        for file in file_list:
            if file in file_contexts:
                # Limit context size to avoid token limits
                context = file_contexts[file]
                contexts[file] = context[:2000] + "..." if len(context) > 2000 else context
        
        # Create a summary of file contexts
        context_summary = "\n\n".join([f"File: {file}\nContext:\n{context}" for file, context in contexts.items()])
        
        # Prompt for the AI to generate test cases
        prompt = f"""
        Generate comprehensive test cases for the '{functionality_name}' functionality in a Java Spring Boot application, which should have full implementation instead of comments.

        Functionality details:
        {json.dumps(functionality, indent=2)}
        
        Context of relevant files:
        {context_summary}
        
        Create two files:
        
        1. A Cucumber feature file (.feature) with:
           - Feature description
           - Background (if needed)
           - Multiple scenarios covering happy paths, error paths, and edge cases
           - Use Gherkin syntax (Given/When/Then)
        
        2. Java step definitions with:
           - All the required @Given, @When, @Then annotations
           - Implementation for each step
           - Appropriate assertions
           - Any required mocks or test data setup
        
        Make sure the steps in the feature file match exactly with those in the step definitions.
        Include comprehensive test coverage for this functionality.
        
        Respond with:
        [FEATURE FILE START]
        Feature: ...
        ...
        [FEATURE FILE END]
        
        [STEP DEFINITIONS START]
        package ...
        ...
        [STEP DEFINITIONS END]
        """
        
        response = client.chat.completions.create(
            messages=[
                {
                    "role": "system", 
                    "content": "You are an expert test automation engineer specializing in BDD testing with Cucumber for Java Spring Boot applications."
                },
                {
                    "role": "user",
                    "content": prompt
                }
            ],
            model="deepseek/deepseek-r1-distill-llama-70b:free",
            stream=False,
        )
        
        # Extract feature file and step definitions from response
        response_text = response.choices[0].message.content
        
        # Parse feature file
        feature_start = response_text.find("[FEATURE FILE START]")
        feature_end = response_text.find("[FEATURE FILE END]")
        
        if feature_start >= 0 and feature_end > feature_start:
            feature_content = response_text[feature_start + len("[FEATURE FILE START]"):feature_end].strip()
            
            # Save feature file
            feature_file_path = os.path.join(test_output_dir, feature_file_name)
            with open(feature_file_path, "w", encoding="utf-8") as f:
                f.write(feature_content)
            print(f"Created feature file: {feature_file_path}")
        
        # Parse step definitions
        step_def_start = response_text.find("[STEP DEFINITIONS START]")
        step_def_end = response_text.find("[STEP DEFINITIONS END]")
        
        if step_def_start >= 0 and step_def_end > step_def_start:
            step_def_content = response_text[step_def_start + len("[STEP DEFINITIONS START]"):step_def_end].strip()
            
            # Save step definitions file
            step_def_file_path = os.path.join(test_output_dir, step_def_file_name)
            with open(step_def_file_path, "w", encoding="utf-8") as f:
                f.write(step_def_content)
            print(f"Created step definitions file: {step_def_file_path}")

In [20]:
def generate_test_summary(functionality_map):
    """
    Generate a summary of all tests created
    """
    functionalities = functionality_map.get("functionalities", [])
    
    summary = "# Test Suite Summary\n\n"
    summary += "| Functionality | Test Priority | Feature File | Step Definitions |\n"
    summary += "|--------------|--------------|-------------|------------------|\n"
    
    for functionality in functionalities:
        name = functionality["name"]
        priority = functionality.get("test_priority", "medium")
        sanitized_name = name.replace(" ", "_").lower()
        feature_file = f"{sanitized_name}.feature"
        step_def_file = f"{sanitized_name}_steps.java"
        
        summary += f"| {name} | {priority} | [{feature_file}](tests/{feature_file}) | [{step_def_file}](tests/{step_def_file}) |\n"
    
    # Save summary
    test_summary_path = os.path.join("./models/output", "test_summary.md")
    with open(test_summary_path, "w", encoding="utf-8") as f:
        f.write(summary)
    print(f"Created test summary: {test_summary_path}")

In [21]:
# Main execution
if __name__ == "__main__":
    print("Starting test generation process...")
    
    # Step 1: Parse context file
    print("Parsing context file...")
    file_contexts = parse_context_file(context_file_path)
    print(f"Found contexts for {len(file_contexts)} files")
    
    # Step 2: Create functionality map
    print("Creating functionality map...")
    functionality_map = create_functionality_map(file_contexts)
    
    # Save functionality map
    with open(functionality_map_path, "w", encoding="utf-8") as f:
        json.dump(functionality_map, f, indent=2)
    print(f"Functionality map saved to {functionality_map_path}")
    
    # Step 3: Generate test cases
    print("Generating test cases...")
    generate_test_cases(functionality_map)
    
    # Step 4: Generate test summary
    print("Generating test summary...")
    generate_test_summary(functionality_map)
    
    print("Test generation completed!")

Starting test generation process...
Parsing context file...
Found contexts for 8 files
Creating functionality map...
Functionality map saved to ./models/context_files/functionality_map.json
Generating test cases...
Generating tests for Account Management...
Generating tests for Transaction Management...
Generating tests for Fraud Detection...
Generating test summary...
Created test summary: ./models/output\test_summary.md
Test generation completed!
